# Modelo del recomedación

## Importación de bases

### Importación de librerías

In [1]:
! pip install cython
! pip install wheel
! pip install --use-pep517 scikit-surprise

  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [60 lines of output]
      <string>:65: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
      !!
      
              ********************************************************************************
              Requirements should be satisfied by a PEP 517 installer.
              If you are using pip, you can try `pip install --use-pep517`.
              ********************************************************************************
      
      !!
      C:\Users\dmon2\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe: No module named pip
      Traceback (most recent call last):
        File "C:\Users\dmon2\AppData\Local\Temp\pip-build-env-w1q93xst\overlay\Lib\site-packages\setuptools\installer.py", line 101, in _fetch_build_egg_no_warn
          subprocess.check_call(cmd)
  

  Using cached scikit-surprise-1.1.3.tar.gz (771 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


In [3]:
# Importación de librerías
import pandas as pd
import numpy as np

#from surprise import SVD
#from surprise import Dataset
#from surprise.model_selection import cross_validate

### Importación de bases de datos

In [4]:
# base = pd.read_parquet("../Data/final/total_generos.parquet")

# Reviews
reviews = pd.read_parquet("../Data/final/reviews_final.parquet")
# Games
games = pd.read_parquet("../Data/final/games_final.parquet")
# items
items = pd.read_parquet("../Data/final/items_final.parquet")


## Creación del modelo de recomendación

In [5]:
r = reviews[["user_id","item_id","sentiment_analysis"]]
r

,user_id,item_id,sentiment_analysis
0,76561197970982479,1250,2
1,76561197970982479,22200,1
4,js41637,227300,2
5,js41637,239030,2
6,evcentric,248820,1
...,...,...,...
57629,76561198312638244,233270,2
57630,76561198312638244,130,2
57631,76561198312638244,70,2
57632,76561198312638244,362890,2


In [6]:
r2 = reviews[["user_id","item_id","recommend"]]
r2

,user_id,item_id,recommend
0,76561197970982479,1250,True
1,76561197970982479,22200,True
4,js41637,227300,True
5,js41637,239030,True
6,evcentric,248820,True
...,...,...,...
57629,76561198312638244,233270,True
57630,76561198312638244,130,True
57631,76561198312638244,70,True
57632,76561198312638244,362890,True
